In [251]:
#!pip install ipywidgets

In [252]:
import requests
import json
import numpy as np
#import tensorflow as tf
import pandas as pd
import os
import pprint
import re
import nbformat
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import csv

from datetime import datetime
from ipywidgets import interact, widgets, fixed, FloatSlider
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [253]:
with open("../DataProcessing/TestData/gameData.json", "r", encoding="utf-8") as file:
    data = json.load(file)
    
with open("../DataProcessing/TestData/postGame.json", "r", encoding="utf-8") as file:
    postData = json.load(file)
    
with open("../DataProcessing/TestData/runeSheet.json", "r", encoding="utf-8") as file:
    runeSheet = json.load(file)
    
participantsData = data['metadata']['participants']
framesData = data['info']['frames']
participantFrames = [frame['participantFrames'] for frame in framesData]

championCountsDict = defaultdict(lambda: defaultdict(int))
namesDict = defaultdict(list)

mostMentionedNamesDict = {}
participantData = []
participantChampionMapping = {}


In [254]:
targetPuuid = '9nFTht795izeoOQoIQMOHtY01trPiRW8SC7kUFIqsDJtTEeNUbpxbRriCUtJ3sApIbpwIBNV44mFjg'
#7hSyI8OKEFPpadVyhqKmTOJlT6umda2FMt1sAeIcZ1WcxdATw7Tvu4ixaR8llqv9u2qPutnAsL9Shg - Jungle
#9nFTht795izeoOQoIQMOHtY01trPiRW8SC7kUFIqsDJtTEeNUbpxbRriCUtJ3sApIbpwIBNV44mFjg - Top

In [255]:
# Function to calculate champion counts
def calculateChampionCounts(data):
    championCountsDict = {}
    namesDict = {}

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))

                    if killerId not in championCountsDict:
                        championCountsDict[killerId] = {}
                        namesDict[killerId] = []

                    if victimId not in championCountsDict:
                        championCountsDict[victimId] = {}
                        namesDict[victimId] = []

                    # Find the killer's and victim's participant data
                    killerNames = [victim.get('name') for victim in event.get('victimDamageReceived', [])]
                    victimNames = [killer.get('name') for killer in event.get('victimDamageDealt', [])]

                    # Use the last name mentioned as the killer name
                    if killerNames:
                        killerName = killerNames[-1]
                        if killerName not in championCountsDict[killerId]:
                            championCountsDict[killerId][killerName] = 1
                        else:
                            championCountsDict[killerId][killerName] += 1
                        namesDict[killerId].append(killerName)

                    # Use the last name mentioned as the victim name
                    if victimNames:
                        victimName = victimNames[-1]
                        if victimName not in championCountsDict[victimId]:
                            championCountsDict[victimId][victimName] = 1
                        else:
                            championCountsDict[victimId][victimName] += 1
                        namesDict[victimId].append(victimName)
    return championCountsDict, namesDict

def calculateMostMentionedChampionNames(data):
    # Calculate champion counts
    championCountsDict, namesDict = calculateChampionCounts(data)

    # Process the counts and choose the most mentioned champion name for each ID
    mostMentionedNamesDict = {}
    for participantId, championCounts in championCountsDict.items():
        mostMentionedChampion = max(championCounts, key=championCounts.get)
        mostMentionedNamesDict[participantId] = mostMentionedChampion
        count = championCounts[mostMentionedChampion]
        names = namesDict[participantId]
    return mostMentionedNamesDict

mostMentionedNamesDict = calculateMostMentionedChampionNames(data)

In [256]:
def addParticipantData(participantId, puuid, championName):  # add championName
    participantData.append([participantId, puuid, championName])  # add championName

def findParticipantData(data):
    if 'info' in data and 'participants' in data['info']:
        for participant in data['info']['participants']:
            participantId = participant['participantId']
            puuid = participant['puuid']
            
            championName = mostMentionedNamesDict[participantId]
            addParticipantData(participantId, puuid, championName)
findParticipantData(data)

def findParticipantByPuuid(participantData, targetPuuid):
    for entry in participantData:
        participantId, puuid, championName = entry
        if puuid == targetPuuid:
            return participantId, championName
    return None, None

targetParticipantId, targetChampionName = findParticipantByPuuid(participantData, targetPuuid)

if targetParticipantId is not None:
    print(f"Participant ID: {targetParticipantId}, Champion: {targetChampionName}")
else:
    print("Participant not found.")

Participant ID: 1, Champion: Jax


In [257]:
def excludeColumns(data, columnsToExclude=None, displayNoZero=False):
    # Exclude specified columns
    excludedData = {}

    for key, value in data.items():
        if key in columnsToExclude:
            continue  # Skip excluded columns
        elif isinstance(value, dict):
            # Recursively exclude columns within nested dictionaries
            excludedData[key] = excludeColumns(value, columnsToExclude, displayNoZero)
        else:
            excludedData[key] = value
    return excludedData

def displayNonZeroParams(participantFrame, displayNoZero=False, indent=''):
    # Iterate through the participant frame's items
    for key, value in participantFrame.items():
        if isinstance(value, dict):
            # If the value is a nested dictionary, iterate through its items
            print(f"{indent}{key} for {targetChampionName}: ")
            displayNonZeroParams(value, displayNoZero, indent + '  ')
        elif isinstance(value, (int, float)) and (not displayNoZero or value != 0):
            # Display the parameter if it's a numeric value and not zero
            print(f"{indent}  {key}: {value}")

def getParticipantDataAtMinute(data, participantId, targetMinute, columnsToExclude=None, displayNoZero=False):
    timestampData = []

    # Check if there are frames in the data
    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            timestamp = frame['timestamp']
            currentMinute = round(timestamp / 60000)  # Convert milliseconds to minutes and round to the nearest minute

            if currentMinute == targetMinute:
                participantFrames = frame.get('participantFrames', {})
                if participantId in participantFrames:
                    participantData = participantFrames[participantId]
                    timestampData.append([timestamp, participantData])

                    # Call the excludeColumns method
                    excludedData = excludeColumns(participantData, columnsToExclude, displayNoZero)

                    # Display either the original data or the excluded data based on your requirement
                    if displayNoZero:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(excludedData, displayNoZero)
                        print("\n---\n")
                    else:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(participantData, displayNoZero)
                        print("\n---\n")
    return timestampData

desiredParticipantId = (f"{targetParticipantId}")
desiredMinute = 2 
columnsToExclude = ['magicDamageDone', 'magicDamageDoneToChampions', 'magicDamageTaken', 
                    'physicalDamageDone', 'physicalDamageDoneToChampions', 'physicalDamageTaken', 
                    'trueDamageDone', 'trueDamageDoneToChampions', 'trueDamageTaken', 'timeEnemySpentControlled']

# Call the function with displayNoZero set to True
#result = getParticipantDataAtMinute(data, desiredParticipantId, desiredMinute, columnsToExclude=columnsToExclude, displayNoZero=True)


In [258]:
def getKillDataForParticipant(data, participantId, participantData):
    killData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))
                    assistIds = [int(assistId) for assistId in event.get('assistingParticipantIds', [])]

                    if killerId == participantId:
                        timestamp = round(event['timestamp'] / 60000, 2)
                        victimDamageReceived = event.get('victimDamageReceived', [])
                        killPosition = event.get('position', {})  # Added line

                        # Calculate total damage done by the killer to the victim
                        totalDamageDone = sum(
                            damageInfo['basic'] + damageInfo['magicDamage'] + damageInfo['physicalDamage'] + damageInfo['trueDamage']
                            for damageInfo in victimDamageReceived
                            if damageInfo['participantId'] == participantId
                        )

                        # Find names from participantData
                        killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                           f"Unknown Champion with ID {killerId}")
                        victimName = next((participant[2] for participant in participantData if participant[0] == victimId),
                                           f"Unknown Champion with ID {victimId}")

                        # Find names of assisting champions and their damage
                        assistingChampions = []
                        damageByAssists = []

                        for assistId in assistIds:
                            assistName = next((participant[2] for participant in participantData if participant[0] == assistId),
                                               f"Unknown Champion with ID {assistId}")

                            assistDamage = sum(
                                damageInfo['basic'] + damageInfo['magicDamage'] + damageInfo['physicalDamage'] + damageInfo['trueDamage']
                                for damageInfo in victimDamageReceived
                                if damageInfo['participantId'] == assistId
                            )

                            assistingChampions.append(assistName)
                            damageByAssists.append(assistDamage)

                        killData.append({
                            'timestamp': timestamp,
                            'killerId': killerId,
                            'killerName': killerName,
                            'victimId': victimId,
                            'victimName': victimName,
                            'totalDamageDone': totalDamageDone,
                            'assistingChampions': assistingChampions,
                            'damageByAssists': damageByAssists,
                            'position': killPosition,
                            'type': 'CHAMPION_KILL'
                        })
    return killData

killDataForParticipant = getKillDataForParticipant(data, targetParticipantId, participantData)

def getAssistsForParticipant(data, targetParticipantId, participantData):
    assistData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    assistIds = [int(assistId) for assistId in event.get('assistingParticipantIds', [])]

                    if targetParticipantId in assistIds:
                        timestamp = round(event['timestamp'] /60000, 2)
                        
                        # Get the name and ID of the killer
                        killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                          f"Unknown Champion with ID {killerId}")

                        killerParticipantId = killerId

                        assistData.append({
                            'timestamp': timestamp,
                            'killerId': killerParticipantId,
                            'killerName': killerName,
                            'assistId': targetParticipantId,
                            'assistName': next((participant[2] for participant in participantData if participant[0] == targetParticipantId),
                                               f"Unknown Champion with ID {targetParticipantId}"),
                            'position': event.get('position', {'x': 0, 'y': 0}),  # Use default coordinates if not present
                            'type': 'CHAMPION_KILL'
                        })
    return assistData

assistDataForParticipant = getAssistsForParticipant(data, targetParticipantId, participantData)

def getDeathsForParticipant(data, targetParticipantId, participantData):
    deathData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    victimId = int(event.get('victimId'))
                    killerId = int(event.get('killerId'))

                    if victimId == targetParticipantId:
                        timestamp = round(event['timestamp'] / 60000, 2)

                        # Get the name and ID of the killer
                        killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                          f"Unknown Champion with ID {killerId}")

                        killerParticipantId = killerId

                        deathData.append({
                            'timestamp': timestamp,
                            'victimId': targetParticipantId,
                            'victimName': next((participant[2] for participant in participantData if participant[0] == targetParticipantId),
                                               f"Unknown Champion with ID {targetParticipantId}"),
                            'killerId': killerParticipantId,
                            'killerName': killerName,
                            'position': event.get('position', {'x': 0, 'y': 0}),  # Use default coordinates if not present
                            'type': 'CHAMPION_KILL'
                        })
    return deathData

#deathLocation = f"X: {death['position'].get('x', 0)}, Y: {death['position'].get('y', 0)}"

deathDataForParticipant = getDeathsForParticipant(data, targetParticipantId, participantData)

In [259]:
for death in deathDataForParticipant:
    # Format the position data
    deathLocation = f"X: {death['position'].get('x', 0)}, Y: {death['position'].get('y', 0)}"
    
    # Print out the details
    print(f"Timestamp: {death['timestamp']} minutes")
    print(f"Victim ID: {death['victimId']}, Victim Name: {death['victimName']}")
    print(f"Killer ID: {death['killerId']}, Killer Name: {death['killerName']}")
    print(f"Death Location: {deathLocation}")
    print(f"Event Type: {death['type']}\n")

Timestamp: 2.92 minutes
Victim ID: 1, Victim Name: Jax
Killer ID: 6, Killer Name: Shen
Death Location: X: 2129, Y: 12160
Event Type: CHAMPION_KILL

Timestamp: 10.39 minutes
Victim ID: 1, Victim Name: Jax
Killer ID: 8, Killer Name: Sylas
Death Location: X: 864, Y: 10706
Event Type: CHAMPION_KILL

Timestamp: 12.25 minutes
Victim ID: 1, Victim Name: Jax
Killer ID: 7, Killer Name: Khazix
Death Location: X: 2050, Y: 11484
Event Type: CHAMPION_KILL

Timestamp: 16.02 minutes
Victim ID: 1, Victim Name: Jax
Killer ID: 6, Killer Name: Shen
Death Location: X: 1352, Y: 9338
Event Type: CHAMPION_KILL

Timestamp: 23.9 minutes
Victim ID: 1, Victim Name: Jax
Killer ID: 9, Killer Name: Ezreal
Death Location: X: 4273, Y: 10043
Event Type: CHAMPION_KILL

Timestamp: 26.05 minutes
Victim ID: 1, Victim Name: Jax
Killer ID: 9, Killer Name: Ezreal
Death Location: X: 4557, Y: 9440
Event Type: CHAMPION_KILL

Timestamp: 32.01 minutes
Victim ID: 1, Victim Name: Jax
Killer ID: 8, Killer Name: Sylas
Death Location:

In [260]:
for assist in assistDataForParticipant:
    assistLocation = f"X: {assist['position'].get('x', 0)}, Y: {assist['position'].get('y', 0)}"

    print(f"Timestamp: {assist['timestamp']}\n"
          f"Killer: {assist['killerName']} (ID: {assist['killerId']})\n"
          f"Assist: {assist['assistName']} (ID: {assist['assistId']})\n"
          f"Assist Location: ({assistLocation})\n")

Timestamp: 17.94
Killer: MasterYi (ID: 2)
Assist: Jax (ID: 1)
Assist Location: (X: 4455, Y: 9895)

Timestamp: 18.1
Killer: Veigar (ID: 3)
Assist: Jax (ID: 1)
Assist Location: (X: 4053, Y: 9372)

Timestamp: 19.31
Killer: MasterYi (ID: 2)
Assist: Jax (ID: 1)
Assist Location: (X: 4071, Y: 8585)

Timestamp: 23.82
Killer: MasterYi (ID: 2)
Assist: Jax (ID: 1)
Assist Location: (X: 4805, Y: 9660)

Timestamp: 23.94
Killer: Varus (ID: 4)
Assist: Jax (ID: 1)
Assist Location: (X: 4190, Y: 9965)

Timestamp: 25.96
Killer: Varus (ID: 4)
Assist: Jax (ID: 1)
Assist Location: (X: 4920, Y: 9304)

Timestamp: 27.82
Killer: Varus (ID: 4)
Assist: Jax (ID: 1)
Assist Location: (X: 9922, Y: 5071)

Timestamp: 27.83
Killer: Veigar (ID: 3)
Assist: Jax (ID: 1)
Assist Location: (X: 9973, Y: 4915)

Timestamp: 27.83
Killer: Veigar (ID: 3)
Assist: Jax (ID: 1)
Assist Location: (X: 9947, Y: 5009)

Timestamp: 34.67
Killer: MasterYi (ID: 2)
Assist: Jax (ID: 1)
Assist Location: (X: 13511, Y: 8141)

Timestamp: 39.04
Killer: 

In [261]:
for kill in killDataForParticipant:
    
    killLocation = f"X: {kill['position'].get('x', 0)}, Y: {kill['position'].get('y', 0)}"

    assistingChampionsLine = f"Assisting Champions: {', '.join(kill['assistingChampions'])}.\n" if kill['assistingChampions'] else ""
    damageByAssistsLine = f"Damage Done by Assists: {', '.join([f'{damage} by {assist}' for damage, assist in zip(kill['damageByAssists'], kill['assistingChampions'])])}.\n" if kill['assistingChampions'] else ""

    print(f"Timestamp: {kill['timestamp']}\n"
          f"Killer: {kill['killerName']} (ID: {kill['killerId']})\n"
          f"Victim: {kill['victimName']} (ID: {kill['victimId']})\n"
          f"Total Damage Done: {kill['totalDamageDone']}\n"
          f"{assistingChampionsLine}"
          f"{damageByAssistsLine}"
          f"Kill Location: ({killLocation})\n")

Timestamp: 4.53
Killer: Jax (ID: 1)
Victim: Shen (ID: 6)
Total Damage Done: 440
Assisting Champions: MasterYi.
Damage Done by Assists: 246 by MasterYi.
Kill Location: (X: 3464, Y: 13074)

Timestamp: 4.73
Killer: Jax (ID: 1)
Victim: Khazix (ID: 7)
Total Damage Done: 833
Kill Location: (X: 3274, Y: 12199)

Timestamp: 8.39
Killer: Jax (ID: 1)
Victim: Shen (ID: 6)
Total Damage Done: 378
Kill Location: (X: 2950, Y: 13188)

Timestamp: 18.03
Killer: Jax (ID: 1)
Victim: Sylas (ID: 8)
Total Damage Done: 721
Assisting Champions: MasterYi, Veigar.
Damage Done by Assists: 1151 by MasterYi, 0 by Veigar.
Kill Location: (X: 4765, Y: 9700)

Timestamp: 25.84
Killer: Jax (ID: 1)
Victim: Khazix (ID: 7)
Total Damage Done: 230
Assisting Champions: Veigar, Varus, Leona.
Damage Done by Assists: 1478 by Veigar, 929 by Varus, 0 by Leona.
Kill Location: (X: 5672, Y: 9180)

Timestamp: 26.01
Killer: Jax (ID: 1)
Victim: Shen (ID: 6)
Total Damage Done: 1276
Assisting Champions: Varus, Leona.
Damage Done by Assists:

In [262]:
def getTakedowns(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            takedowns = participant.get('challenges', {}).get('takedowns', None)
            if takedowns is not None:
                print(f"  Takedowns by {targetChampionName}: {takedowns}")
            else:
                print(f"No Data for Takedowns by {targetChampionName}")
    return None

def getKda(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            kda = participant.get('challenges', {}).get('kda', None)
            if kda is not None:
                print(f"  KDA by {targetChampionName}: {kda}\n")
            else:
                print(f"No KDA found for {targetChampionName}\n")
    return None

def getSoloKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            soloKills = participant.get('challenges', {}).get('soloKills', None)
            if soloKills is not None:
                print(f"  Solo Kills for {targetChampionName}: {soloKills}")
            else:
                print(f"No kill participation found for {targetChampionName}")
    return None
    
def getKillParticipation(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            killParticipation = participant.get('challenges', {}).get('killParticipation', None)
            if killParticipation is not None:
                print(f"  Kill Participation for {targetChampionName}: {killParticipation}")
            else:
                print(f"No kill participation found for {targetChampionName}")
            return killParticipation
    return None    

def getLaneRole(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            laneRole = participant.get('individualPosition', None)
            if laneRole is not None:
                print(f"Role of {targetChampionName}: {laneRole}")
            return laneRole
    return None  
    
def getJungleStats(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid and participant.get('individualPosition') == 'JUNGLE':
            totalAllyJungleMinionsKilled = participant.get('totalAllyJungleMinionsKilled', None)
            totalEnemyJungleMinionsKilled = participant.get('totalEnemyJungleMinionsKilled', None)
            if totalAllyJungleMinionsKilled is not None and totalEnemyJungleMinionsKilled is not None:
                print(f"  Total Ally Jungle Minions Killed: {totalAllyJungleMinionsKilled}")
                print(f"  Total Enemy Jungle Minions Killed: {totalEnemyJungleMinionsKilled}\n")
            return totalAllyJungleMinionsKilled, totalEnemyJungleMinionsKilled
    return None, None

def getTeamSide(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            teamSide = participant.get('teamId', None)
            if teamSide is not None:
                if teamSide == 100:
                    print(f"{targetChampionName} was Blue Side")
                else:
                    print(f"{targetChampionName} was Red Side")
            return teamSide
    return None

def getGameWon(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            gameWon = participant.get('win', None)
            if gameWon is not None:
                if gameWon == True:
                    print(f"{targetChampionName}s Team Won the Game")
                else:
                    print(f"{targetChampionName}s Team Lost the Game")        
            return
    return None

def getTotalDamageDealt(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            totalDamageDealt = participant.get('totalDamageDealt', None)
            if totalDamageDealt is not None:
                print(f"  Total Damage Dealt by {targetChampionName}: {totalDamageDealt}")
    return None

def getTotalDamageDealtToChampions(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            totalDamageDealtToChampions = participant.get('totalDamageDealtToChampions', None)
            if totalDamageDealtToChampions is not None:
                print(f"  Total Damage Dealt To Champions by {targetChampionName}: {totalDamageDealtToChampions}")
    return None

def getTotalDamageTaken(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            totalDamageTaken = participant.get('totalDamageTaken', None)
            if totalDamageTaken is not None:
                print(f"  Total Damage Taken for {targetChampionName}: {totalDamageTaken}\n")
    return None

def getDeathsByEnemyChamps(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            deathsByEnemyChamps =  participant.get('challenges', {}).get('deathsByEnemyChamps', None)
            if deathsByEnemyChamps is not None:
                print(f"  Deaths by Enemy Champions on {targetChampionName}: {deathsByEnemyChamps}")
    return None

def getTeamBaronKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            teamBaronKills = participant.get('challenges', {}).get('teamBaronKills', None)
            if teamBaronKills is not None and teamBaronKills != 0:
                print(f"  Team Baron Kills by {targetChampionName}: {teamBaronKills}")
    return None

def getTeamRiftHeraldKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            teamRiftHeraldKills = participant.get('challenges', {}).get('teamRiftHeraldKills', None)
            if teamRiftHeraldKills is not None and teamRiftHeraldKills != 0:
                print(f"  Team Rift Herald Kills by {targetChampionName}: {teamRiftHeraldKills}")
    return None

def getTeamElderDragonKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            teamElderDragonKills = participant.get('challenges', {}).get('teamElderDragonKills', None)
            if teamElderDragonKills is not None and teamElderDragonKills != 0:
                print(f"  Team Elder Dragon Kills by {targetChampionName}: {teamElderDragonKills}")
    return None

def getDragonTakedowns(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            dragonTakedowns = participant.get('challenges', {}).get('dragonTakedowns', None)
            if dragonTakedowns is not None and dragonTakedowns != 0:
                print(f"  Dragon Takedowns by {targetChampionName}: {dragonTakedowns}")
    return None

def getVisionScoreAdvantageLaneOpponent(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            visionScoreAdvantage = participant.get('challenges', {}).get('visionScoreAdvantageLaneOpponent')
            if visionScoreAdvantage is not None:
                print(f"  Vision Score Advantage for {targetChampionName}: {visionScoreAdvantage}")
    return None

def getTurretTakedowns(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            turretTakedowns = participant.get('challenges', {}).get('turretTakedowns', None)
            if turretTakedowns is not None and turretTakedowns != 0:
                print(f"  Turret Takedowns by {targetChampionName}: {turretTakedowns}\n")  
    return None

def getVisionScore(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            visionScore = participant.get('visionScore', None)
            if visionScore is not None:
                print(f"  Vison Score for {targetChampionName}: {visionScore}")
    return None

def getControlWardTimeCoverageInRiverOrEnemyHalf(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            controlWardTimeCoverageInRiverOrEnemyHalf = participant.get('challenges', {}).get('controlWardTimeCoverageInRiverOrEnemyHalf', None)
            if controlWardTimeCoverageInRiverOrEnemyHalf is not None and controlWardTimeCoverageInRiverOrEnemyHalf != 0:
                print(f"  Control Ward Time Coverage In River or Enemy Half for {targetChampionName}: {controlWardTimeCoverageInRiverOrEnemyHalf}")
    return None

def getControlWardsPlaced(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            controlWardsPlaced = participant.get('challenges', {}).get('controlWardsPlaced', None)
            if controlWardsPlaced is not None and controlWardsPlaced != 0:
                print(f"  Control Wards placed by {targetChampionName}: {controlWardsPlaced}")
    return None

def getStealthWardsPlaced(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            stealthWardsPlaced = participant.get('challenges', {}).get('stealthWardsPlaced', None)
            if stealthWardsPlaced is not None and stealthWardsPlaced != 0:
                print(f"  Stealth Wards Placed by {targetChampionName}: {stealthWardsPlaced}")
    return None

def getWardsKilled(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            wardsKilled = participant.get('wardsKilled', None)
            if wardsKilled is not None and wardsKilled != 0:
                print(f"  Wards Killed by {targetChampionName}: {wardsKilled}\n")
    return None

def getWardsPlaced(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            wardsPlaced = participant.get('wardsPlaced', None)
            if wardsPlaced is not None and wardsPlaced != 0:
                print(f"  Wards Placed by {targetChampionName}: {wardsPlaced}")
    return None

def getLaneMinionsFirst10Minutes(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            laneMinionsFirst10Minutes = participant.get('challenges', {}).get('laneMinionsFirst10Minutes', None)
            if laneMinionsFirst10Minutes is not None and getLaneRole != "JUNGLE":
                print(f"  Lane Minions First 10 Minutes by {targetChampionName}: {laneMinionsFirst10Minutes}\n") 
    return None

def getGoldPerMinute(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            goldPerMinute = participant.get('challenges', {}).get('goldPerMinute', None)
            if goldPerMinute is not None:
                print(f"  Gold Per Minute for {targetChampionName}: {goldPerMinute}")
    return None

getLaneRole(postData, targetPuuid)
getJungleStats(postData, targetPuuid)

getSoloKills(postData, targetPuuid)
getKillParticipation(postData, targetPuuid)
getTakedowns(postData, targetPuuid)
getDeathsByEnemyChamps(postData, targetPuuid)
getKda(postData, targetPuuid)

getTotalDamageDealt(postData, targetPuuid)
getTotalDamageDealtToChampions(postData, targetPuuid)
getTotalDamageTaken(postData, targetPuuid)

getTeamBaronKills(postData, targetPuuid)
getTeamRiftHeraldKills(postData, targetPuuid)
getDragonTakedowns(postData, targetPuuid)
getTeamElderDragonKills(postData, targetPuuid)
getTurretTakedowns(postData, targetPuuid)

getVisionScore(postData, targetPuuid)
getVisionScoreAdvantageLaneOpponent(postData, targetPuuid)

getWardsPlaced(postData, targetPuuid)
getStealthWardsPlaced(postData, targetPuuid)
getControlWardsPlaced(postData, targetPuuid)
getControlWardTimeCoverageInRiverOrEnemyHalf(postData, targetPuuid)
getWardsKilled(postData, targetPuuid)

getGoldPerMinute(postData,targetPuuid)
getLaneMinionsFirst10Minutes(postData, targetPuuid)

getTeamSide(postData, targetPuuid)
getGameWon(postData, targetPuuid)

Role of Jax: TOP
  Solo Kills for Jax: 3
  Kill Participation for Jax: 0.42
  Takedowns by Jax: 21
  Deaths by Enemy Champions on Jax: 9
  KDA by Jax: 2.3333333333333335

  Total Damage Dealt by Jax: 210367
  Total Damage Dealt To Champions by Jax: 28986
  Total Damage Taken for Jax: 41089

  Team Baron Kills by Jax: 1
  Team Elder Dragon Kills by Jax: 1
  Turret Takedowns by Jax: 5

  Vison Score for Jax: 18
  Vision Score Advantage for Jax: -0.3503682613372803
  Wards Placed by Jax: 7
  Stealth Wards Placed by Jax: 7
  Wards Killed by Jax: 4

  Gold Per Minute for Jax: 454.0979780906803
  Lane Minions First 10 Minutes by Jax: 58

Jax was Blue Side
Jaxs Team Won the Game


In [263]:
def getEventsInMinute(data, targetMinute):
    eventsInMinute = []
    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            timestamp = frame['timestamp']
            currentMinute = round(timestamp / 60000, 2)  # Round to two decimal places
            if currentMinute == targetMinute:
                eventsInMinute.extend(frame.get('events', []))
    return eventsInMinute

In [264]:
def getTowerDestroyedData(data, participantData):
    towerData = []  # Rename the variable for consistency

    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'BUILDING_KILL':
                teamId = event['teamId']
                towerType = event.get('towerType', None)
                timestamp = event['timestamp']
                currentMinute = round(timestamp / 60000, 2)
                position = event['position']
                laneType = event['laneType']
                killerId = event['killerId']
                assistingParticipantIds = event.get('assistingParticipantIds', [])

                # Retrieve champion name for the killer
                killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                  f"Minions")

                # Retrieve champion names for assisting participants only if there are assisting participants
                assistingChampionNames = [next((participant[2] for participant in participantData if participant[0] == assistId),
                                               f"Unknown Champion with ID {assistId}") for assistId in assistingParticipantIds]

                towerType = towerType if towerType in ['OUTER_TURRET', 'INNER_TURRET'] else None
                
                towerData.append({
                    'teamId': teamId,
                    'towerType': towerType,  # Keep towerType as it is
                    'timestamp': currentMinute,
                    'position': position,
                    'laneType': laneType,
                    'killerId': killerId,
                    'killerName': killerName,
                    'assistingParticipantIds': assistingParticipantIds,
                    'assistingChampionNames': assistingChampionNames,
                    'type': 'BUILDING_KILL'
                })

    return towerData

# Example usage:
towerData = getTowerDestroyedData(data, participantData)

sortedTowerData = sorted(towerData, key=lambda x: (x['teamId'], x['timestamp']))

# Display the extracted data
for tower in sortedTowerData:
    currentMinute = tower['timestamp']
    towerType = tower['towerType']
    
    # Check if towerType is 'OUTER_TURRET' or 'INNER_TURRET' before printing
    if towerType in ['OUTER_TURRET', 'INNER_TURRET']:
        print(f"Team: {tower['teamId']}, Tower Type: {towerType}, Timestamp: {currentMinute}, Lane: {tower['laneType']}")
        print(f"Killer: {tower['killerName']}")
    
        # Print assisting champions only if there are assisting participants
        if tower['assistingParticipantIds']:
            print(f"Assisting Champions: {tower['assistingChampionNames']}")
    
        print('\n')

Team: 100, Tower Type: OUTER_TURRET, Timestamp: 20.04, Lane: BOT_LANE
Killer: Ezreal
Assisting Champions: ['Nautilus']


Team: 100, Tower Type: OUTER_TURRET, Timestamp: 20.91, Lane: MID_LANE
Killer: Sylas


Team: 100, Tower Type: OUTER_TURRET, Timestamp: 29.4, Lane: TOP_LANE
Killer: Minions


Team: 100, Tower Type: INNER_TURRET, Timestamp: 29.9, Lane: TOP_LANE
Killer: Minions
Assisting Champions: ['Shen']


Team: 200, Tower Type: OUTER_TURRET, Timestamp: 17.34, Lane: MID_LANE
Killer: Veigar


Team: 200, Tower Type: OUTER_TURRET, Timestamp: 20.36, Lane: TOP_LANE
Killer: Jax


Team: 200, Tower Type: INNER_TURRET, Timestamp: 21.02, Lane: TOP_LANE
Killer: Jax


Team: 200, Tower Type: OUTER_TURRET, Timestamp: 24.95, Lane: BOT_LANE
Killer: MasterYi


Team: 200, Tower Type: INNER_TURRET, Timestamp: 28.63, Lane: MID_LANE
Killer: Varus


Team: 200, Tower Type: INNER_TURRET, Timestamp: 37.34, Lane: BOT_LANE
Killer: Minions




In [265]:
gameStartTimestamp = postData['info']['gameStartTimestamp']
gameEndTimestamp = postData['info']['gameEndTimestamp']
gameDurationMs = gameEndTimestamp - gameStartTimestamp
gameDurationMinutes = gameDurationMs / 60000

In [266]:
def plotEventsAtMinute(backgroundImagePath, killData, assistData, deathData, towerData, targetMinute, title):
    plt.figure()
    plt.title(f'{title} at Minute {targetMinute}')

    # Load background image
    backgroundImage = mpimg.imread(backgroundImagePath)
    plt.xlim(0, 15000)
    plt.ylim(0, 15000)
    plt.imshow(backgroundImage, extent=[0, 15000, 0, 15000], alpha=0.8)

    # Combine kill, assist, death, and tower data
    allEvents = killData + assistData + deathData + towerData

    # Filter events for the specified minute
    minuteEvents = [event for event in allEvents if event['timestamp'] <= targetMinute]
    
    # Print kills, assists, and deaths for the target participant in the specified minute
    targetKills = [event for event in minuteEvents if event.get('type') == 'CHAMPION_KILL' and event.get('killerId') == targetParticipantId]
    targetAssists = [event for event in minuteEvents if event.get('type') == 'CHAMPION_KILL' and event.get('assistId') == targetParticipantId]
    targetDeaths = [event for event in minuteEvents if event.get('type') == 'CHAMPION_KILL' and event.get('victimId') == targetParticipantId]
    towerDestroyed = [event for event in minuteEvents if event.get('type') == 'BUILDING_KILL' and event.get('towerType') in ['OUTER_TURRET', 'INNER_TURRET']]

    print(f'\nKills for {targetChampionName} in minute {targetMinute}:')
    for kill in targetKills:
        print(f"Timestamp: {kill['timestamp']}, Victim: {kill.get('victimName', 'Unknown')}")

    print(f'\nAssists for {targetChampionName} in minute {targetMinute}:')
    for assist in targetAssists:
        print(f"Timestamp: {assist['timestamp']}, Killer: {assist.get('killerName', 'Unknown')}")

    print(f'\nDeaths for {targetChampionName} in minute {targetMinute}:')
    for death in targetDeaths:
        print(f"Timestamp: {death['timestamp']}, Killer: {death.get('killerName', 'Unknown')}")

    print(f'\nTowers Destroyed in minute {targetMinute}:')
    for tower in towerDestroyed:
        x = tower['position']['x']
        y = tower['position']['y']
        print(f"Timestamp: {tower['timestamp']}, Killer: {tower['killerName']}, Tower Position: ({x}, {y})")  # Print tower position for debugging
        plt.scatter(x, y, color='yellow', marker='d')

    # Scatter plot for kills, assists, and deaths
    for event in minuteEvents:
        x = event['position']['x']
        y = event['position']['y']

        if event.get('type') == 'CHAMPION_KILL' and event.get('killerId') == targetParticipantId:
            plt.scatter(x, y, color='blue', marker='o')
        elif event.get('type') == 'CHAMPION_KILL' and event.get('assistId') == targetParticipantId:
            plt.scatter(x, y, color='green', marker='*')
        elif event.get('type') == 'CHAMPION_KILL' and event.get('victimId') == targetParticipantId:
            plt.scatter(x, y, color='red', marker='X')

    plt.show()


# Example usage:
# Assuming killDataForParticipant, assistDataForParticipant, deathDataForParticipant,
# and towerDataForParticipant contain the relevant events for kills, assists, deaths,
# and towers destroyed for the target participant
# You can specify the minute you want to see

interact(
    plotEventsAtMinute,
    backgroundImagePath=widgets.fixed('../DataProcessing/TestData/lolMap.PNG'),
    killData=widgets.fixed(killDataForParticipant),
    assistData=widgets.fixed(assistDataForParticipant),
    deathData=widgets.fixed(deathDataForParticipant),
    towerData=widgets.fixed(towerData),
    targetMinute=widgets.FloatSlider(value=21, min=0, max=gameDurationMinutes, step=1, description='Minute:', readout_format='.1f'),
    title=widgets.fixed('KDA Locations')
)

interactive(children=(FloatSlider(value=21.0, description='Minute:', max=39.13223333333333, readout_format='.1…

<function __main__.plotEventsAtMinute(backgroundImagePath, killData, assistData, deathData, towerData, targetMinute, title)>

In [267]:
def extractBuildingKillData(data, participantData):
    buildingKillData = []
    turret_combinations = {}  # Store turret combinations and their corresponding cases

    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'BUILDING_KILL':
                teamId = event['teamId']
                towerType = event.get('towerType', 'Unknown Tower Type')
                laneType = event['laneType']

                # Combine relevant information into a key
                turret_key = (teamId, towerType, laneType)

                if turret_key not in turret_combinations:
                    # Assign a new case if the combination is encountered for the first time
                    turret_combinations[turret_key] = len(turret_combinations) + 1

                case = turret_combinations[turret_key]

                timestamp = event['timestamp']
                currentMinute = round(timestamp / 60000, 2)
                position = event['position']
                killerId = event['killerId']
                assistingParticipantIds = event.get('assistingParticipantIds', [])

                # Retrieve champion name for the killer
                killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                  f"Unknown Champion with ID {killerId}")

                # Retrieve champion names for assisting participants only if there are assisting participants
                assistingChampionNames = [next((participant[2] for participant in participantData if participant[0] == assistId),
                                               f"Unknown Champion with ID {assistId}") for assistId in assistingParticipantIds]

                buildingKillData.append({
                    'case': case,
                    'teamId': teamId,
                    'towerType': towerType,
                    'laneType': laneType,
                    'timestamp': currentMinute,
                    'position': position,
                    'killerId': killerId,
                    'killerName': killerName,
                    'assistingParticipantIds': assistingParticipantIds,
                    'assistingChampionNames': assistingChampionNames
                })

    return buildingKillData

# Example usage:
buildingKillData = extractBuildingKillData(data, participantData)

# Display the extracted data
for kill in buildingKillData:
    print(f"Case: {kill['case']}")
    print(f"Team: {kill['teamId']}, Tower Type: {kill['towerType']}, Lane: {kill['laneType']}, Timestamp: {kill['timestamp']}")
    print(f"Killer: {kill['killerName']}")
    
    # Print assisting champions only if there are assisting participants
    if kill['assistingParticipantIds']:
        print(f"Assisting Champions: {kill['assistingChampionNames']}")
    
    print('\n')

Case: 1
Team: 200, Tower Type: OUTER_TURRET, Lane: MID_LANE, Timestamp: 17.34
Killer: Veigar


Case: 2
Team: 100, Tower Type: OUTER_TURRET, Lane: BOT_LANE, Timestamp: 20.04
Killer: Ezreal
Assisting Champions: ['Nautilus']


Case: 3
Team: 200, Tower Type: OUTER_TURRET, Lane: TOP_LANE, Timestamp: 20.36
Killer: Jax


Case: 4
Team: 100, Tower Type: OUTER_TURRET, Lane: MID_LANE, Timestamp: 20.91
Killer: Sylas


Case: 5
Team: 200, Tower Type: INNER_TURRET, Lane: TOP_LANE, Timestamp: 21.02
Killer: Jax


Case: 6
Team: 200, Tower Type: OUTER_TURRET, Lane: BOT_LANE, Timestamp: 24.95
Killer: MasterYi


Case: 7
Team: 200, Tower Type: INNER_TURRET, Lane: MID_LANE, Timestamp: 28.63
Killer: Varus


Case: 8
Team: 100, Tower Type: OUTER_TURRET, Lane: TOP_LANE, Timestamp: 29.4
Killer: Unknown Champion with ID 0


Case: 9
Team: 100, Tower Type: INNER_TURRET, Lane: TOP_LANE, Timestamp: 29.9
Killer: Unknown Champion with ID 0
Assisting Champions: ['Shen']


Case: 10
Team: 200, Tower Type: BASE_TURRET, Lane

In [268]:
#blue turrets
bto=True #BLUE TOP OUTER
bti=True #BLUE TOP INNER
bmo=False #BLUE MID OUTER
bmi=False #BLUE MID INNER
bbo=True #BLUE BOT OUTER
bbi=True #BLUE BOT INNER

In [269]:
#red turrets
rto=True #RED TOP OUTER
rti=True #RED TOP INNER
rmo=True #RED MID OUTER
rmi=True #RED MID INNER
rbo=True #RED BOT OUTER
rbi=True #RED BOT INNER

In [270]:
def resetTurrets():
    global bto, bti, bmo, bmi, bbo, bbi, rto, rti, rmo, rmi, rbo, rbi
    bto=True #BLUE TOP OUTER
    bti=True #BLUE TOP INNER
    bmo=True #BLUE MID OUTER
    bmi=True #BLUE MID INNER
    bbo=True #BLUE BOT OUTER
    bbi=True #BLUE BOT INNER
    rto=True #RED TOP OUTER
    rti=True #RED TOP INNER
    rmo=True #RED MID OUTER
    rmi=True #RED MID INNER
    rbo=True #RED BOT OUTER
    rbi=True #RED BOT INNER

In [271]:
def blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi):
    # Initialize caseNumber to 0
    caseNumber = 0
    
    # Check for each case and set the caseNumber accordingly
    if bto and bti and bmo and bmi and bbo and bbi:
        caseNumber = 1
    elif not bto and not bti and not bmo and not bmi and not bbo and not bbi:
        caseNumber = 2
    elif not bmo and not bmi and bto and bti and bbo and bbi:
        caseNumber = 3
    elif not bmo and bmi and bto and bti and bbo and bbi:
        caseNumber = 4
    elif not bto and bti and bmo and bmi and bbo and bbi:
        caseNumber = 5
    elif not bto and not bti and bmo and bmi and bbo and bbi:
        caseNumber = 6
    elif not bbo and bto and bti and bbi and bmo and bmi:
        caseNumber = 7
    elif not bbo and not bbi and bto and bti and bmo and bmi:
        caseNumber = 8
    elif not bmo and not bbo and bto and bti and bmi and bbi:
        caseNumber = 9
    elif not bto and not bmo and bti and bmi and bbo and bbo:
        caseNumber = 10
    elif not bto and not bbo and bbi and bti and bmo and bmi:
        caseNumber = 11
    elif not bto and not bmo and not bbo and bti and bmi and bbi:
        caseNumber = 12
    elif not bbo and not bmo and not bmi and bbi and bto and bti:
        caseNumber = 13
    elif not bto and not bmo and not bmi and bti and bbo and bbi:
        caseNumber = 14
    elif not bmo and not bbo and not bbi and bmi and bto and bti:
        caseNumber = 15
    elif not bto and not bbo and not bbi and bti and bmo and bmi:
        caseNumber = 16
    elif not bto and not bti and not bmo and bmi and bbo and bbi:
        caseNumber = 17
    elif not bto and not bti and not bbo and bbi and bmo and bmi:
        caseNumber = 18
    elif bti and bbi and not bto and not bbo and not bmo and not bmi:
        caseNumber = 19
    elif bmi and bbi and not bmo and not bbo and not bto and not bti:
        caseNumber = 20
    elif bti and bmi and not bto and not bmo and not bbo and not bbi:
        caseNumber = 21
    elif bbo and bbi and not bto and not bti and not bmo and not bmi:
        caseNumber = 22
    elif bto and bti and not bmo and not bmi and not bbo and not bbi:
        caseNumber = 23
    elif bmo and bmi and not bto and not bti and not bbo and not bbi:
        caseNumber = 24
    elif bbi and not bbo and not bti and not bto and not bmo and not bmi: 
        caseNumber = 25
    elif bti and not bto and not bmo and not bmi and not bbo and not bbi:
        caseNumber = 26
    elif bmi and not bmo and not bto and not bti and not bbo and not bbi:
        caseNumber = 27
    else:
        caseNumber=404
    
    return caseNumber

In [272]:
# Test scenarios for blueDefCaseCalc function

# Case 1: All variables are True
bto = bti = bmo = bmi = bbo = bbi = True
print("Case 1:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 1

# Case 2: All variables are False
bto = bti = bmo = bmi = bbo = bbi = False
print("Case 2:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 2

# Case 3: bmo and bmi are False, others are True
bto = bti = bbo = bbi = True
bmo = bmi = False
print("Case 3:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 3

# Case 4: bmo is False, others are True
bto = bti = bbo = bbi = True
bmo = False
print("Case 4:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 4

# Case 5: bto is False, others are True
bti = bmo = bmi = bbo = bbi = True
bto = False
print("Case 5:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 5

# Case 6: bto and bti are False, others are True
bmo = bmi = bbo = bbi = True
bto = bti = False
print("Case 6:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 6

# Case 7: bbo is False, others are True
bto = bti = bmo = bmi = bbi = True
bbo = False
print("Case 7:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 7

# Case 8: bbo and bbi are False, others are True
bto = bti = bmo = bmi = True
bbo = bbi = False
print("Case 8:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 8

# Case 9: bmo and bbo are False, others are True
bto = bti = bmi = bbi = True
bmo = bbo = False
print("Case 9:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 9

# Case 10: bto and bmo are False, others are True
bti = bmi = bbo = bbi = True
bto = bmo = False
print("Case 10:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 10

# Case 11: bto and bbo are False, others are True
bti = bmi = bmo = bbi = True
bto = bbo = False
print("Case 11:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 11

# Case 12: bto, bmo, and bbo are False, others are True
bti = bmi = bbi = True
bto = bmo = bbo = False
print("Case 12:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 12

# Case 13: bbo, bmo, and bmi are False, others are True
bto = bti = bbi = True
bbo = bmo = bmi = False
print("Case 13:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 13

# Case 14: bto, bmo, and bmi are False, others are True
bti = bbo = bbi = True
bto = bmo = bmi = False
print("Case 14:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 14

# Case 15: bmo, bbo, and bbi are False, others are True
bto = bti = bmi = True
bmo = bbo = bbi = False
print("Case 15:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 15

# Case 16: bto, bbo, and bbi are False, others are True
bti = bmi = bmo = True
bto = bbo = bbi = False
print("Case 16:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 16

# Case 17: bto, bti, and bmo are False, others are True
bbo = bmi = bbi = True
bto = bti = bmo = False
print("Case 17:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 17

# Case 18: bto, bti, and bbo are False, others are True
bmo = bmi = bbi = True
bto = bti = bbo = False
print("Case 18:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 18

# Case 19: bti and bbi are True, others are False
bto = bmo = bmi = bbo = False
bti = bbi = True
print("Case 19:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 19

# Case 20: bmi and bbi are True, others are False
bto = bti = bmo = bbo = False
bmi = bbi = True
print("Case 20:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 20

# Case 21: bti and bmi are True, others are False
bto = bmo = bbo = bbi = False
bti = bmi = True
print("Case 21:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 21

# Case 22: bbo and bbi are True, others are False
bto = bti = bmo = bmi = False
bbo = bbi = True
print("Case 22:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 22

# Case 23: bto and bti are True, others are False
bmo = bmi = bbo = bbi = False
bto = bti = True
print("Case 23:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 23

# Case 24: bmo and bmi are True, others are False
bto = bti = bbo = bbi = False
bmo = bmi = True
print("Case 24:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 24

# Case 25: bbi is True, others are False
bto = bti = bmo = bmi = bbo = False
bbi = True
print("Case 25:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 25

# Case 26: bti is True, others are False
bto = bmo = bmi = bbo = bbi = False
bti = True
print("Case 26:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 26

# Case 27: bmi is True, others are False
bto = bti = bmo = bbo = bbi = False
bmi = True
print("Case 27:", blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi))  # Expected output: 27

Case 1: 1
Case 2: 2
Case 3: 3
Case 4: 3
Case 5: 5
Case 6: 6
Case 7: 7
Case 8: 8
Case 9: 9
Case 10: 10
Case 11: 11
Case 12: 12
Case 13: 13
Case 14: 14
Case 15: 15
Case 16: 16
Case 17: 17
Case 18: 18
Case 19: 19
Case 20: 20
Case 21: 21
Case 22: 22
Case 23: 23
Case 24: 24
Case 25: 25
Case 26: 26
Case 27: 27


In [273]:
def redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi):
    # Initialize caseNumber to 0
    caseNumber = 0
    
    # Check for each case and set the caseNumber accordingly
    if rto and rti and rmo and rmi and rbo and rbi:
        caseNumber = 1
    elif not rto and not rti and not rmo and not rmi and not rbo and not rbi:
        caseNumber = 2
    elif not rmo and not rmi and rto and rti and rbo and rbi:
        caseNumber = 3
    elif not rmo and rmi and rto and rti and rbo and rbi:
        caseNumber = 4
    elif not rto and rti and rmo and rmi and rbo and rbi:
        caseNumber = 5
    elif not rto and not rti and rmo and rmi and rbo and rbi:
        caseNumber = 6
    elif not rbo and rto and rti and rbi and rmo and rmi:
        caseNumber = 7
    elif not rbo and not rbi and rto and rti and rmo and rmi:
        caseNumber = 8
    elif not rmo and not rbo and rto and rti and rmi and rbi:
        caseNumber = 9
    elif not rto and not rmo and rti and rmi and rbo and rbo:
        caseNumber = 10
    elif not rto and not rbo and rbi and rti and rmo and rmi:
        caseNumber = 11
    elif not rto and not rmo and not rbo and rti and rmi and rbi:
        caseNumber = 12
    elif not rbo and not rmo and not rmi and rbi and rto and rti:
        caseNumber = 13
    elif not rto and not rmo and not rmi and rti and rbo and rbi:
        caseNumber = 14
    elif not rmo and not rbo and not rbi and rmi and rto and rti:
        caseNumber = 15
    elif not rto and not rbo and not rbi and rti and rmo and rmi:
        caseNumber = 16
    elif not rto and not rti and not rmo and rmi and rbo and rbi:
        caseNumber = 17
    elif not rto and not rti and not rbo and rbi and rmo and rmi:
        caseNumber = 18
    elif rti and rbi and not rto and not rbo and not rmo and not rmi:
        caseNumber = 19
    elif rmi and rbi and not rmo and not rbo and not rto and not rti:
        caseNumber = 20
    elif rti and rmi and not rto and not rmo and not rbo and not rbi:
        caseNumber = 21
    elif rbo and rbi and not rto and not rti and not rmo and not rmi:
        caseNumber = 22
    elif rto and rti and not rmo and not rmi and not rbo and not rbi:
        caseNumber = 23
    elif rmo and rmi and not rto and not rti and not rbo and not rbi:
        caseNumber = 24
    elif rbi and not rbo and not rti and not rto and not rmo and not rmi: 
        caseNumber = 25
    elif rti and not rto and not rmo and not rmi and not rbo and not rbi:
        caseNumber = 26
    elif rmi and not rmo and not rto and not rti and not rbo and not rbi:
        caseNumber = 27
    else:
        caseNumber = 404
    
    return caseNumber

In [274]:
# Test scenarios for redDefCaseCalc function

# Case 1: All variables are True
rto = rti = rmo = rmi = rbo = rbi = True
print("Case 1:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 1

# Case 2: All variables are False
rto = rti = rmo = rmi = rbo = rbi = False
print("Case 2:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 2

# Case 3: rmo and rmi are False, others are True
rto = rti = rbo = rbi = True
rmo = rmi = False
print("Case 3:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 3

# Case 4: rmo is False, others are True
rto = rti = rbo = rbi = True
rmo = False
print("Case 4:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 4

# Case 5: rto is False, others are True
rti = rmo = rmi = rbo = rbi = True
rto = False
print("Case 5:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 5

# Case 6: rto and rti are False, others are True
rmo = rmi = rbo = rbi = True
rto = rti = False
print("Case 6:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 6

# Case 7: rbo is False, others are True
rto = rti = rmo = rmi = rbi = True
rbo = False
print("Case 7:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 7

# Case 8: rbo and rbi are False, others are True
rto = rti = rmo = rmi = True
rbo = rbi = False
print("Case 8:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 8

# Case 9: rmo and rbo are False, others are True
rto = rti = rmi = rbi = True
rmo = rbo = False
print("Case 9:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 9

# Case 10: rto and rmo are False, others are True
rti = rmi = rbo = rbi = True
rto = rmo = False
print("Case 10:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 10

# Case 11: rto and rbo are False, others are True
rti = rmi = rmo = rbi = True
rto = rbo = False
print("Case 11:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 11

# Case 12: rto, rmo, and rbo are False, others are True
rti = rmi = rbi = True
rto = rmo = rbo = False
print("Case 12:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 12

# Case 13: rbo, rmo, and rmi are False, others are True
rto = rti = rbi = True
rbo = rmo = rmi = False
print("Case 13:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 13

# Case 14: rto, rmo, and rmi are False, others are True
rti = rbo = rbi = True
rto = rmo = rmi = False
print("Case 14:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 14

# Case 15: rmo, rbo, and rbi are False, others are True
rto = rti = rmi = True
rmo = rbo = rbi = False
print("Case 15:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 15

# Case 16: rto, rbo, and rbi are False, others are True
rti = rmi = rmo = True
rto = rbo = rbi = False
print("Case 16:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 16

# Case 17: rto, rti, and rmo are False, others are True
rbo = rmi = rbi = True
rto = rti = rmo = False
print("Case 17:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 17

# Case 18: rto, rti, and rbo are False, others are True
rmo = rmi = rbi = True
rto = rti = rbo = False
print("Case 18:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 18

# Case 19: rti and rbi are True, others are False
rto = rmo = rmi = rbo = False
rti = rbi = True
print("Case 19:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 19

# Case 20: rmi and rbi are True, others are False
rto = rti = rmo = rbo = False
rmi = rbi = True
print("Case 20:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 20

# Case 21: rti and rmi are True, others are False
rto = rmo = rbo = rbi = False
rti = rmi = True
print("Case 21:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 21

# Case 22: rbo and rbi are True, others are False
rto = rti = rmo = rmi = False
rbo = rbi = True
print("Case 22:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 22

# Case 23: rto and rti are True, others are False
rmo = rmi = rbo = rbi = False
rto = rti = True
print("Case 23:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 23

# Case 24: rmo and rmi are True, others are False
rto = rti = rbo = rbi = False
rmo = rmi = True
print("Case 24:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 24

# Case 25: rbi is True, others are False
rto = rti = rmo = rmi = rbo = False
rbi = True
print("Case 25:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 25

# Case 26: rti is True, others are False
rto = rmo = rmi = rbo = rbi = False
rti = True
print("Case 26:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 26

# Case 27: rmi is True, others are False
rto = rti = rmo = rbo = rbi = False
rmi = True
print("Case 27:", redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi))  # Expected output: 27

Case 1: 1
Case 2: 2
Case 3: 3
Case 4: 3
Case 5: 5
Case 6: 6
Case 7: 7
Case 8: 8
Case 9: 9
Case 10: 10
Case 11: 11
Case 12: 12
Case 13: 13
Case 14: 14
Case 15: 15
Case 16: 16
Case 17: 17
Case 18: 18
Case 19: 19
Case 20: 20
Case 21: 21
Case 22: 22
Case 23: 23
Case 24: 24
Case 25: 25
Case 26: 26
Case 27: 27


In [275]:
# Dictionary to store timestamps for each team
blueTimestamps = {}
redTimestamps = {}

In [276]:
def resetTowerData():
    # Reset blue_timestamps dictionary
    global blueTimestamps, redTimestamps
    blueTimestamps.clear()
    blueTimestamps[1] = 0

    # Reset red_timestamps dictionary
    redTimestamps.clear()
    redTimestamps[1] = 0

In [277]:
resetTowerData()

print("Blue Team Timestamps:")
for caseNumber, timestamp in blueTimestamps.items():
    print(f"Case Number: {caseNumber}, Timestamp: {timestamp}")

# Print contents of red_timestamps dictionary
print("Red Team Timestamps:")
for caseNumber, timestamp in redTimestamps.items():
    print(f"Case Number: {caseNumber}, Timestamp: {timestamp}")

Blue Team Timestamps:
Case Number: 1, Timestamp: 0
Red Team Timestamps:
Case Number: 1, Timestamp: 0


In [278]:
def updateBlueTowerData(caseNumber, timestamp):
    global blueTimestamps
    blueTimestamps[caseNumber] = timestamp

def updateRedTowerData(caseNumber, timestamp):
    global redTimestamps
    redTimestamps[caseNumber] = timestamp

def getTowerDestroyedData(data, participantData):
    towerData = []
    caseBlue = 0
    caseRed = 0

    # Initialize boolean variables for blue team
    bto = bti = bmo = bmi = bbo = bbi = True

    # Initialize boolean variables for red team
    rto = rti = rmo = rmi = rbo = rbi = True
    
    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'BUILDING_KILL':
                teamId = event['teamId']
                towerType = event.get('towerType', None)
                laneType = event['laneType']
                timestamp = event['timestamp']
                
                # Update boolean variables based on tower destruction events
                if towerType in ['OUTER_TURRET', 'INNER_TURRET']:
                    if teamId == 100:
                        if towerType == 'OUTER_TURRET':
                            if laneType == 'TOP_LANE':
                                bto = False  # blue top outer destroyed
                            elif laneType == 'MID_LANE':
                                bmo = False  # blue mid outer destroyed
                            elif laneType == 'BOT_LANE':
                                bbo = False  # blue bot outer destroyed
                        elif towerType == 'INNER_TURRET':
                            if laneType == 'TOP_LANE':
                                bti = False  # blue top inner destroyed
                            elif laneType == 'MID_LANE':
                                bmi = False  # blue mid inner destroyed
                            elif laneType == 'BOT_LANE':
                                bbi = False  # blue bot inner destroyed
                        # Call blueDefCaseCalc with updated boolean variables
                        caseBlue = blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi)
                        # Update blue team timestamps
                        updateBlueTowerData(caseBlue, timestamp)
                    elif teamId == 200:
                        if towerType == 'OUTER_TURRET':
                            if laneType == 'TOP_LANE':
                                rto = False  # red top outer destroyed
                            elif laneType == 'MID_LANE':
                                rmo = False  # red mid outer destroyed
                            elif laneType == 'BOT_LANE':
                                rbo = False  # red bot outer destroyed
                        elif towerType == 'INNER_TURRET':
                            if laneType == 'TOP_LANE':
                                rti = False  # red top inner destroyed
                            elif laneType == 'MID_LANE':
                                rmi = False  # red mid inner destroyed
                            elif laneType == 'BOT_LANE':
                                rbi = False  # red bot inner destroyed
                        # Call redDefCaseCalc with updated boolean variables
                        caseRed = redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi)
                        # Update red team timestamps
                        updateRedTowerData(caseRed, timestamp)

                # Append tower data to the list
                towerData.append({
                    'teamId': teamId,
                    'towerType': towerType,
                    'laneType': laneType,
                    'timestamp': timestamp
                })

    return towerData

# Example usage:
towerData = getTowerDestroyedData(data, participantData)

# Display the extracted data
for tower in towerData:
    teamId = tower['teamId']
    towerType = tower['towerType']
    laneType = tower['laneType']
    timestamp = tower['timestamp']

    if towerType in ['OUTER_TURRET', 'INNER_TURRET']:
        print(f"Team: {teamId}, Tower Type: {towerType}, Lane: {laneType}, timestamp {timestamp}")
        
print("Blue Team Timestamps:")
for caseNumber, timestamp in blueTimestamps.items():
    print(f"Case Number: {caseNumber}, Timestamp: {timestamp}")

# Print contents of redTimestamps dictionary
print("Red Team Timestamps:")
for caseNumber, timestamp in redTimestamps.items():
    print(f"Case Number: {caseNumber}, Timestamp: {timestamp}")

Team: 200, Tower Type: OUTER_TURRET, Lane: MID_LANE, timestamp 1040566
Team: 100, Tower Type: OUTER_TURRET, Lane: BOT_LANE, timestamp 1202540
Team: 200, Tower Type: OUTER_TURRET, Lane: TOP_LANE, timestamp 1221698
Team: 100, Tower Type: OUTER_TURRET, Lane: MID_LANE, timestamp 1254851
Team: 200, Tower Type: INNER_TURRET, Lane: TOP_LANE, timestamp 1261413
Team: 200, Tower Type: OUTER_TURRET, Lane: BOT_LANE, timestamp 1496800
Team: 200, Tower Type: INNER_TURRET, Lane: MID_LANE, timestamp 1717741
Team: 100, Tower Type: OUTER_TURRET, Lane: TOP_LANE, timestamp 1763842
Team: 100, Tower Type: INNER_TURRET, Lane: TOP_LANE, timestamp 1793760
Team: 200, Tower Type: INNER_TURRET, Lane: BOT_LANE, timestamp 2240232
Blue Team Timestamps:
Case Number: 1, Timestamp: 0
Case Number: 7, Timestamp: 1202540
Case Number: 9, Timestamp: 1254851
Case Number: 12, Timestamp: 1763842
Case Number: 20, Timestamp: 1793760
Red Team Timestamps:
Case Number: 1, Timestamp: 0
Case Number: 4, Timestamp: 1040566
Case Number:

In [279]:
%run sectionFunctions.ipynb

Safe , threat = 0
Danger , threat = 100
Danger , threat = 100
Danger , threat = 100
Safe , threat = 0
Danger , threat = 100
Safe , threat = 0
Safe , threat = 0
Safe , threat = 0
Danger , threat = 100
Safe , threat = 0
Danger , threat = 100
Danger , threat = 100
Safe , threat = 0
Safe , threat = 0
Safe , threat = 0
Safe , threat = 0
Safe , threat = 0
Safe , threat = 0
Safe , threat = 0
Safe , threat = 0
Safe , threat = 0
Safe , threat = 0
Safe , threat = 0
Safe , threat = 0
Danger , threat = 100
Danger , threat = 100
Danger , threat = 100
Danger , threat = 100
Safe , threat = 0
Safe , threat = 0
Danger , threat = 100
Danger , threat = 100
Safe , threat = 0
Safe , threat = 0
Danger , threat = 100
Danger , threat = 100
Safe , threat = 0
Danger , threat = 100
Danger , threat = 100
Safe , threat = 0
Danger , threat = 100
Danger , threat = 100
Danger , threat = 100
Danger , threat = 100
Safe , threat = 0
Safe , threat = 0
Safe , threat = 0
Danger , threat = 100
Danger , threat = 100
Danger ,

In [280]:
def findClosestCaseNumber(blueTimestamps, redTimestamps, participantId, eventTimestamp):
    # Determine which timestamps dictionary to use based on participantId
    timestamps = blueTimestamps if participantId > 0 and participantId < 6 else redTimestamps
    
    closestCaseNumber = None
    smallestDifference = float('inf')
    
    for caseNumber, timestamp in timestamps.items():
        difference = abs(timestamp - eventTimestamp)
        if difference < smallestDifference:
            smallestDifference = difference
            closestCaseNumber = caseNumber
    
    return closestCaseNumber

In [281]:
def executeClosestCaseFunction(blueTimestamps, redTimestamps, participantId, eventTimestamp, x, y):
    # Use the modified findClosestCaseNumber function
    closestCaseNumber = findClosestCaseNumber(blueTimestamps, redTimestamps, participantId, eventTimestamp)
    
    # Determine the function name prefix based on participantId
    functionPrefix = "blueCase" if participantId > 0 and participantId < 6 else "redCase"
    
    # Construct the function name
    functionName = f"{functionPrefix}{closestCaseNumber}"
    
    # Retrieve the function object using globals() or locals() if the function is defined in a local scope
    functionToCall = globals().get(functionName)
    
    # Check if the function exists
    if functionToCall:
        # Print the name of the function being called
        print(f"Calling function: {functionName}")
        
        # Call the function with x and y, and return its result
        return functionToCall(x, y)
    else:
        return "Function does not exist."

In [282]:
eventTimestamp=1254852
participantId=4
x = 5100
y = 5100

In [283]:
executeClosestCaseFunction(blueTimestamps, redTimestamps, participantId, eventTimestamp, x, y)

Calling function: blueCase9
Safe , threat = 0


0

In [284]:
#2nd version with incorporated threat
def getDeathsForParticipant(data, targetParticipantId, participantData, blue_timestamps, red_timestamps):
    deathData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    victimId = int(event.get('victimId'))
                    if victimId == targetParticipantId:
                        timestamp = round(event['timestamp'] / 60000, 2)  # Convert timestamp to minutes for consistency
                        
                        x = event.get('position', {}).get('x', 0)
                        y = event.get('position', {}).get('y', 0)

                        # Determine safety status at the time of death
                        safetyStatus = executeClosestCaseFunction(
                            blue_timestamps, red_timestamps, targetParticipantId, event['timestamp'], x, y
                        )

                        killerId = int(event.get('killerId'))
                        killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                          f"Unknown Champion with ID {killerId}")
                        victimName = next((participant[2] for participant in participantData if participant[0] == targetParticipantId),
                                          f"Unknown Champion with ID {targetParticipantId}")

                        deathData.append({
                            'timestamp': timestamp,
                            'victimId': targetParticipantId,
                            'victimName': victimName,
                            'killerId': killerId,
                            'killerName': killerName,
                            'position': {'x': x, 'y': y},
                            'type': 'CHAMPION_KILL',
                            'safetyStatus': safetyStatus  # Include safety status in the death data
                        })
    return deathData

In [285]:
def writeDeathDataToCSV(deathData, headers=None):
    """
    Writes death data to a CSV file at a fixed file path.

    Parameters:
    - deathData: A list of dictionaries containing death event data.
    - headers: An optional list of headers for the CSV file. If not provided, a default set will be used.
    """
    # Hardcoded file path
    filePath = '../DataProcessing/TestData/participant_deathdata.csv'

    # Default headers if not provided
    if headers is None:
        headers = ['timestamp', 'victimId', 'victimName', 'killerId', 'killerName', 'position_x', 'position_y', 'type', 'safetyStatus']

    # Ensure the directory exists
    os.makedirs(os.path.dirname(filePath), exist_ok=True)

    # Write data to the CSV file
    with open(filePath, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        for death in deathData:
            # Flatten the position data for CSV
            death['position_x'] = death['position']['x']
            death['position_y'] = death['position']['y']
            del death['position']  # Adjust if necessary for your data structure
            writer.writerow(death)

    print(f"Data has been written to '{filePath}'.")
deathDataForParticipant = getDeathsForParticipant(data, targetParticipantId, participantData, blueTimestamps, redTimestamps)
writeDeathDataToCSV(deathDataForParticipant)

Calling function: blueCase1
Safe , threat = 0
Calling function: blueCase7
Safe , threat = 0
Calling function: blueCase7
Safe , threat = 0
Calling function: blueCase7
Safe , threat = 0
Calling function: blueCase9
Danger , threat = 100
Calling function: blueCase12
Danger , threat = 100
Calling function: blueCase20
Danger , threat = 100
Calling function: blueCase20
Danger , threat = 100
Calling function: blueCase20
Danger , threat = 100
Data has been written to '../DataProcessing/TestData/participant_deathdata.csv'.


In [286]:
#2nd version with incorporated threat
def getAssistsForParticipant(data, targetParticipantId, participantData, blue_timestamps, red_timestamps):
    assistData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    assistIds = [int(assistId) for assistId in event.get('assistingParticipantIds', [])]

                    if targetParticipantId in assistIds:
                        timestamp = round(event['timestamp'] / 60000, 2)
                        x = event.get('position', {}).get('x', 0)
                        y = event.get('position', {}).get('y', 0)

                        # Determine safety status at the time of the assist
                        safetyStatus = executeClosestCaseFunction(
                            blueTimestamps, redTimestamps, targetParticipantId, event['timestamp'], x, y
                        )

                        killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                          f"Unknown Champion with ID {killerId}")

                        assistData.append({
                            'timestamp': timestamp,
                            'killerId': killerId,
                            'killerName': killerName,
                            'assistId': targetParticipantId,
                            'assistName': next((participant[2] for participant in participantData if participant[0] == targetParticipantId),
                                               f"Unknown Champion with ID {targetParticipantId}"),
                            'position': {'x': x, 'y': y},
                            'type': 'CHAMPION_KILL',
                            'safetyStatus': safetyStatus
                        })
    return assistData


In [287]:
def writeAssistDataToCSV(assistData, headers=None):
    """
    Writes assist data to a CSV file at a fixed file path.

    Parameters:
    - assistData: A list of dictionaries containing assist event data.
    - headers: An optional list of headers for the CSV file. If not provided, a default set will be used.
    """

    # Hardcoded file path
    filePath = '../DataProcessing/TestData/participant_assistdata.csv'

    # Default headers if not provided
    if headers is None:
        headers = ['timestamp', 'killerId', 'killerName', 'assistId', 'assistName', 'position_x', 'position_y', 'type', 'safetyStatus']

    # Ensure the directory exists
    os.makedirs(os.path.dirname(filePath), exist_ok=True)

    # Write data to the CSV file
    with open(filePath, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        for assist in assistData:
            # Flatten the position data for CSV
            assist['position_x'] = assist['position']['x']
            assist['position_y'] = assist['position']['y']
            del assist['position']  # Adjust if necessary for your data structure
            writer.writerow(assist)

    print(f"Data has been written to '{filePath}'.")

assistDataForParticipant = getAssistsForParticipant(data, targetParticipantId, participantData, blueTimestamps, redTimestamps)
writeAssistDataToCSV(assistDataForParticipant)

Calling function: blueCase7
Safe , threat = 0
Calling function: blueCase7
Safe , threat = 0
Calling function: blueCase7
Safe , threat = 0
Calling function: blueCase9
Danger , threat = 100
Calling function: blueCase9
Danger , threat = 100
Calling function: blueCase12
Danger , threat = 100
Calling function: blueCase12
Danger , threat = 100
Calling function: blueCase12
Danger , threat = 100
Calling function: blueCase12
Danger , threat = 100
Calling function: blueCase20
Danger , threat = 100
Calling function: blueCase20
Danger , threat = 100
Data has been written to '../DataProcessing/TestData/participant_assistdata.csv'.


In [288]:
#2nd version with incorporated threat
def getKillDataForParticipant(data, participantId, participantData, blueTimestamps, redTimestamps):
    killData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))

                    if killerId == participantId:
                        timestamp = round(event['timestamp'] / 60000, 2)
                        x = event.get('position', {}).get('x', 0)
                        y = event.get('position', {}).get('y', 0)

                        # Determine safety status at the time of the kill
                        safetyStatus = executeClosestCaseFunction(
                            blueTimestamps, redTimestamps, participantId, event['timestamp'], x, y
                        )

                        victimDamageReceived = event.get('victimDamageReceived', [])
                        totalDamageDone = sum(
                            damage['basic'] + damage['magicDamage'] + damage['physicalDamage'] + damage['trueDamage']
                            for damage in victimDamageReceived
                            if damage['participantId'] == participantId
                        )

                        killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                          f"Unknown Champion with ID {killerId}")
                        victimName = next((participant[2] for participant in participantData if participant[0] == victimId),
                                          f"Unknown Champion with ID {victimId}")

                        killData.append({
                            'timestamp': timestamp,
                            'killerId': killerId,
                            'killerName': killerName,
                            'victimId': victimId,
                            'victimName': victimName,
                            'totalDamageDone': totalDamageDone,
                            'position': {'x': x, 'y': y},
                            'type': 'CHAMPION_KILL',
                            'safetyStatus': safetyStatus
                        })
    return killData

In [289]:
killDataForParticipant = getKillDataForParticipant(data, targetParticipantId, participantData, blueTimestamps, redTimestamps)

Calling function: blueCase1
Danger , threat = 100
Calling function: blueCase1
Danger , threat = 100
Calling function: blueCase1
Danger , threat = 100
Calling function: blueCase7
Safe , threat = 0
Calling function: blueCase12
Danger , threat = 100
Calling function: blueCase12
Danger , threat = 100
Calling function: blueCase12
Danger , threat = 100
Calling function: blueCase20
Danger , threat = 100
Calling function: blueCase20
Danger , threat = 100
Calling function: blueCase20
Danger , threat = 100


In [290]:
def writeKillDataToCSV(killData, headers=None):
    """
    Writes kill data to a CSV file at a fixed file path.

    Parameters:
    - killData: A list of dictionaries containing kill event data.
    - headers: An optional list of headers for the CSV file. If not provided, a default set will be used.
    """

    # Hardcoded file path
    filePath = '../DataProcessing/TestData/participant_killdata.csv'

    # Default headers if not provided
    if headers is None:
        headers = ['timestamp', 'killerId', 'killerName', 'victimId', 'victimName', 'totalDamageDone', 'position_x', 'position_y', 'type', 'safetyStatus']

    # Ensure the directory exists
    os.makedirs(os.path.dirname(filePath), exist_ok=True)

    # Write data to the CSV file
    with open(filePath, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        for kill in killData:
            # Flatten the position data for CSV
            kill['position_x'] = kill['position']['x']
            kill['position_y'] = kill['position']['y']
            del kill['position']  # Adjust if necessary for your data structure
            writer.writerow(kill)

    print(f"Data has been written to '{filePath}'.")
    
killDataForParticipant = getKillDataForParticipant(data, targetParticipantId, participantData, blueTimestamps, redTimestamps)
writeKillDataToCSV(killDataForParticipant)

Calling function: blueCase1
Danger , threat = 100
Calling function: blueCase1
Danger , threat = 100
Calling function: blueCase1
Danger , threat = 100
Calling function: blueCase7
Safe , threat = 0
Calling function: blueCase12
Danger , threat = 100
Calling function: blueCase12
Danger , threat = 100
Calling function: blueCase12
Danger , threat = 100
Calling function: blueCase20
Danger , threat = 100
Calling function: blueCase20
Danger , threat = 100
Calling function: blueCase20
Danger , threat = 100
Data has been written to '../DataProcessing/TestData/participant_killdata.csv'.


In [291]:
filePath = '../DataProcessing/TestData/participant_deathdata.csv'
deathDataDf = pd.read_csv(filePath)

# Display the first few rows to verify the data
print(deathDataDf.head(15))

# Calculate the average timestamp of deaths
averageDeathThreat = deathDataDf['safetyStatus'].mean()

print(f"The average threat of deaths is {averageDeathThreat}.")


   timestamp  victimId victimName  killerId killerName  position_x  \
0       2.92         1        Jax         6       Shen        2129   
1      10.39         1        Jax         8      Sylas         864   
2      12.25         1        Jax         7     Khazix        2050   
3      16.02         1        Jax         6       Shen        1352   
4      23.90         1        Jax         9     Ezreal        4273   
5      26.05         1        Jax         9     Ezreal        4557   
6      32.01         1        Jax         8      Sylas        8722   
7      34.79         1        Jax         7     Khazix       13683   
8      37.84         1        Jax         8      Sylas       13848   

   position_y           type  safetyStatus  
0       12160  CHAMPION_KILL             0  
1       10706  CHAMPION_KILL             0  
2       11484  CHAMPION_KILL             0  
3        9338  CHAMPION_KILL             0  
4       10043  CHAMPION_KILL           100  
5        9440  CHAMPION_KILL 

In [292]:
def getTowerDestroyedData(data):
    towerData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                # Check for building kills that are specifically tower destructions
                if event['type'] == 'BUILDING_KILL' and event.get('buildingType') == 'TOWER_BUILDING':
                    timestamp = round(event['timestamp'] / 60000, 2)  # Convert timestamp to minutes
                    x = event.get('position', {}).get('x', 0)
                    y = event.get('position', {}).get('y', 0)
                    towerType = event.get('towerType', 'Unknown Tower Type')
                    laneType = event.get('laneType', 'Unknown Lane')
                    teamId = event.get('teamId', 'Unknown Team')
                    killerId = event.get('killerId', 0)  # Use 0 or some placeholder for environmental kills or unattributed
                    
                    # List of assisting participant IDs
                    assistingParticipantIds = event.get('assistingParticipantIds', [])

                    towerData.append({
                        'timestamp': timestamp,
                        'position_x': x,
                        'position_y': y,
                        'towerType': towerType,
                        'laneType': laneType,
                        'teamId': teamId,
                        'killerId': killerId,
                        'assistingParticipantIds': assistingParticipantIds,  # Storing list of IDs directly
                        'type': 'BUILDING_KILL'
                    })

    return pd.DataFrame(towerData)

In [293]:
def writeTowerDataToCSV(towerData_df, filePath='../DataProcessing/TestData/tower_destruction_data.csv'):
    """
    Writes tower destruction data to a CSV file.

    Parameters:
    - towerData_df: A pandas DataFrame containing tower destruction event data.
    - filePath: Path to the file where the CSV will be written. Defaults to a fixed path.
    """

    # Ensure the directory exists
    os.makedirs(os.path.dirname(filePath), exist_ok=True)

    # Write data to the CSV file
    towerData_df.to_csv(filePath, index=False)

    print(f"Data has been written to '{filePath}'.")

# Assuming 'data' and 'participantData' contain your match and participant information
towerData_df = getTowerDestroyedData(data)
print(type(towerData_df))

# Call the function to write the DataFrame to a CSV file
writeTowerDataToCSV(towerData_df)


<class 'pandas.core.frame.DataFrame'>
Data has been written to '../DataProcessing/TestData/tower_destruction_data.csv'.


In [294]:
filePath = '../DataProcessing/TestData/tower_destruction_data.csv'
towerDestroyedDf = pd.read_csv(filePath)

# Display the first few rows to verify the data
print(towerDestroyedDf.head())

   timestamp  position_x  position_y     towerType  laneType  teamId  \
0      17.34        8955        8510  OUTER_TURRET  MID_LANE     200   
1      20.04       10504        1029  OUTER_TURRET  BOT_LANE     100   
2      20.36        4318       13875  OUTER_TURRET  TOP_LANE     200   
3      20.91        5846        6396  OUTER_TURRET  MID_LANE     100   
4      21.02        7943       13411  INNER_TURRET  TOP_LANE     200   

   killerId assistingParticipantIds           type  
0         3                      []  BUILDING_KILL  
1         9                    [10]  BUILDING_KILL  
2         1                      []  BUILDING_KILL  
3         8                      []  BUILDING_KILL  
4         1                      []  BUILDING_KILL  


In [295]:
def calculateSafetyStatusMean():
    # Paths to the CSV files
    filePaths = {
        'death': '../DataProcessing/TestData/participant_deathdata.csv',
        'assist': '../DataProcessing/TestData/participant_assistdata.csv',
        'kill': '../DataProcessing/TestData/participant_killdata.csv'
    }
    
    # Load the data
    dataFrames = {key: pd.read_csv(path) for key, path in filePaths.items()}
    
    # Calculate and print mean safetyStatus for each file
    for key, df in dataFrames.items():
        meanSafetyStatus = df['safetyStatus'].mean()
        print(f"Mean safetyStatus for {key}: {meanSafetyStatus}")
    
    # Combined mean for assists and kills
    combinedAssistKill = pd.concat([dataFrames['assist'], dataFrames['kill']])
    combinedMeanAssistKill = combinedAssistKill['safetyStatus'].mean()
    print(f"Combined mean safetyStatus for assists and kills: {combinedMeanAssistKill}")
    
    # Combined mean for all three datasets
    allCombined = pd.concat(dataFrames.values())
    combinedMeanAll = allCombined['safetyStatus'].mean()
    print(f"Combined mean safetyStatus for all three datasets: {combinedMeanAll}")

# Call the function
calculateSafetyStatusMean()


Mean safetyStatus for death: 55.55555555555556
Mean safetyStatus for assist: 72.72727272727273
Mean safetyStatus for kill: 90.0
Combined mean safetyStatus for assists and kills: 80.95238095238095
Combined mean safetyStatus for all three datasets: 73.33333333333333


In [296]:
def totalDamageDoneByParticipant(victimDamageReceived, participantId):
    return sum(damage['magicDamage'] + damage['physicalDamage'] + damage['trueDamage']
               for damage in victimDamageReceived if damage['participantId'] == participantId)


In [297]:
# threat is showing killers and assisters threat, not the victim
def processAndWriteGameDataToCsv(filePath, outputCsvPath):
    # Load JSON data
    with open(filePath, 'r') as file:
        data = json.load(file)

    # Define CSV headers
    headers = ['timestamp', 'eventType', 'participantId', 'participantName', 'positionX', 'positionY', 'dangerScore', 'totalDamageDone', 'victimId']

    # Prepare for CSV writing
    os.makedirs(os.path.dirname(outputCsvPath), exist_ok=True)
    with open(outputCsvPath, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()

        # Process data
        for frame in data['info']['frames']:
            for event in frame['events']:
                if event['type'] == 'CHAMPION_KILL':
                    # Common event data
                    timestamp = round(event['timestamp'] / 60000, 2)
                    positionX = event.get('position', {}).get('x', 0)
                    positionY = event.get('position', {}).get('y', 0)
                    # Process killer
                    killerId = event.get('killerId')
                    victimId = event.get('victimId')

                    safetyStatus = executeClosestCaseFunction(blueTimestamps, redTimestamps, killerId, event['timestamp'], positionX, positionY)
                    if killerId:  # If killerId is present and not 0
                        damageDone = totalDamageDoneByParticipant(event.get('victimDamageReceived', []), killerId)
                        writer.writerow({
                            'timestamp': timestamp,
                            'eventType': 'Kill',
                            'participantId': killerId,
                            'participantName': next((participant[2] for participant in participantData if participant[0] == killerId), f"Unknown Champion with ID {killerId}"),
                            'positionX': positionX,
                            'positionY': positionY,
                            'dangerScore': safetyStatus,  # Placeholder for safetyStatus logic
                            'totalDamageDone': damageDone,  # Placeholder for damage calculation logic
                            'victimId': victimId
                        })
                    # Process assists
                    for assistId in event.get('assistingParticipantIds', []):
                        damageDone = totalDamageDoneByParticipant(event.get('victimDamageReceived', []), assistId)
                        writer.writerow({
                            'timestamp': timestamp,
                            'eventType': 'Assist',
                            'participantId': assistId,
                            'participantName': next((participant[2] for participant in participantData if participant[0] == assistId), f"Unknown Champion with ID {assistId}"),
                            'positionX': positionX,
                            'positionY': positionY,
                            'dangerScore': safetyStatus,  # Placeholder for safetyStatus logic
                            'totalDamageDone': damageDone,  # Blank for assists as it's not directly applicable
                            'victimId': victimId
                        })

if __name__ == "__main__":
    inputFilePath = '../DataProcessing/TestData/gameData.json'
    outputCsvPath = '../DataProcessing/TestData/gameKillsAssists.csv'
    processAndWriteGameDataToCsv(inputFilePath, outputCsvPath)


Calling function: redCase1
Danger , threat = 100
Calling function: redCase1
Danger , threat = 100
Calling function: blueCase1
Danger , threat = 100
Calling function: blueCase1
Danger , threat = 100
Calling function: redCase1
Safe , threat = 0
Calling function: blueCase1
Danger , threat = 100
Calling function: redCase1
Safe , threat = 0
Calling function: blueCase1
Danger , threat = 100
Calling function: redCase4
Danger , threat = 100
Calling function: blueCase1
Danger , threat = 100
Calling function: blueCase1
Danger , threat = 100
Calling function: redCase4
Danger , threat = 100
Calling function: redCase4
Safe , threat = 0
Calling function: redCase4
Danger , threat = 100
Calling function: blueCase7
Danger , threat = 100
Calling function: redCase4
Danger , threat = 100
Calling function: blueCase7
Danger , threat = 100
Calling function: blueCase7
Danger , threat = 100
Calling function: redCase4
Danger , threat = 100
Calling function: redCase4
Danger , threat = 100
Calling function: redCa

In [298]:
file_path = '../DataProcessing/TestData/gameKillsAssists.csv'
killsAssistsDataDf = pd.read_csv(file_path)
killsAssistsDataDf.head(50)

,timestamp,eventType,participantId,participantName,positionX,positionY,dangerScore,totalDamageDone,victimId
0,2.92,Kill,6,Shen,2129,12160,100,404,1
1,4.48,Kill,7,Khazix,2200,12018,100,606,2
2,4.48,Assist,6,Shen,2200,12018,100,269,2
3,4.53,Kill,1,Jax,3464,13074,100,438,6
4,4.53,Assist,2,MasterYi,3464,13074,100,245,6
5,4.73,Kill,1,Jax,3274,12199,100,831,7
6,6.41,Kill,7,Khazix,9477,6446,0,970,2
7,6.41,Kill,2,MasterYi,9400,6528,100,996,7
8,6.65,Kill,8,Sylas,10934,4356,0,797,5
9,8.39,Kill,1,Jax,2950,13188,100,377,6


In [299]:
killsAssistsDataDf.describe()

,timestamp,participantId,positionX,positionY,dangerScore,totalDamageDone,victimId
count,223.000000,223.000000,223.000000,223.000000,223.000000,223.000000,223.000000
mean,24.698610,4.834081,8167.174888,7708.565022,88.789238,787.995516,5.860987
std,8.797522,2.639667,3434.537332,3669.340353,31.620860,653.707800,2.891967
min,2.920000,1.000000,864.000000,870.000000,0.000000,0.000000,1.000000
25%,20.195000,3.000000,4765.000000,4888.500000,100.000000,312.500000,3.000000
50%,25.240000,5.000000,9477.000000,8109.000000,100.000000,585.000000,6.000000
75%,30.080000,7.000000,10297.000000,9988.500000,100.000000,1171.500000,8.000000
max,39.040000,10.000000,13848.000000,13823.000000,100.000000,3505.000000,10.000000


In [305]:
def plotEventsAtMinute(backgroundImagePath, killDataPath, assistDataPath, deathDataPath, targetMinute, towerDestroyedPath, title):
    plt.figure(figsize=(10, 10))
    plt.title(f'{title} at Minute {targetMinute}')
    
    # Load background image
    backgroundImage = mpimg.imread(backgroundImagePath)
    plt.xlim(0, 15000)
    plt.ylim(0, 15000)
    plt.imshow(backgroundImage, extent=[0, 15000, 0, 15000], alpha=0.8)
    
    # Load kill, assist, and death data from CSV files
    killData = pd.read_csv(killDataPath)
    assistData = pd.read_csv(assistDataPath)
    deathData = pd.read_csv(deathDataPath)
    towerData = pd.read_csv(towerDestroyedPath)

    # Filter events for the specified minute
    killEvents = killData[(killData['timestamp'] <= targetMinute) & (killData['killerId'] == targetParticipantId)]
    assistEvents = assistData[(assistData['timestamp'] <= targetMinute) & (assistData['assistId'] == targetParticipantId)]
    deathEvents = deathData[(deathData['timestamp'] <= targetMinute) & (deathData['victimId'] == targetParticipantId)]
    
    towerDestroyed = towerData[(towerData['timestamp'] <= targetMinute) & 
                               (towerData['type'] == 'BUILDING_KILL') & 
                               (towerData['towerType'].isin(['OUTER_TURRET', 'INNER_TURRET']))]

    # Print kills, assists, and deaths
    printEvents('Kill', killEvents, 'victimName', targetMinute)
    printEvents('Assist', assistEvents, 'killerName', targetMinute) 
    printEvents('Death', deathEvents, 'killerName', targetMinute)
    printTowerDestructions(towerDestroyed)
    
    TPI = lambda event: event['killerId'] == targetParticipantId or targetParticipantId in [int(x) for x in event['assistingParticipantIds'].strip("[]").split(", ") if x.isdigit()]


    plotOnMap(killEvents, 'position_x', 'position_y', 'blue', 'o')
    plotOnMap(assistEvents, 'position_x', 'position_y', 'green', '*')
    plotOnMap(deathEvents,  'position_x', 'position_y', 'red', 'x')
    plotOnMap(towerDestroyed, 'position_x', 'position_y', 'yellow', 'd', additionalCondition=TPI, alternateColor='darkorange')
    
    plt.show()
    
def printEvents(eventType, events, targetColumn, targetMinute):
    print(f'\n{eventType}s for {targetChampionName} in minute {targetMinute}:')
    for _, event in events.iterrows():
        safetyStatus = "Danger" if event['safetyStatus'] == 100 else "Safe" if event['safetyStatus'] == 0 else "Unknown"
        print(f"Timestamp: {event['timestamp']}, {eventType}: {event[targetColumn]}, Safety Status: {safetyStatus}")
    
def plotOnMap(events, posX, posY, color, marker, additionalCondition=None, alternateColor=None):
    for _, event in events.iterrows():
        # Check if an additional condition is provided and if it's true
        if additionalCondition and additionalCondition(event):
            currentColor = alternateColor
        else:
            currentColor = color
        plt.scatter(event[posX], event[posY], color=currentColor, marker=marker, s=100)


def printTowerDestructions(towerEvents):
    print('\nTowers Destroyed:')
    for _, event in towerEvents.iterrows():
        print(f"Timestamp: {event['timestamp']}, Team ID: {event['teamId']}, Tower Type: {event['towerType']}, Lane: {event['laneType']}, Position: ({event['position_x']}, {event['position_y']})")
        
interact(
    plotEventsAtMinute,
    backgroundImagePath=fixed('../DataProcessing/TestData/lolMap.PNG'),
    killDataPath=fixed('../DataProcessing/TestData/participant_killdata.csv'),
    assistDataPath=fixed('../DataProcessing/TestData/participant_assistdata.csv'),
    deathDataPath=fixed('../DataProcessing/TestData/participant_deathdata.csv'),
    towerDestroyedPath=fixed('../DataProcessing/TestData/tower_destruction_data.csv'),
    targetMinute=FloatSlider(value=21, min=0, max=45, step=1, description='Minute:', readout_format='.1f'),  # Adjust 'max' as per your game duration
    title=fixed('KDA Locations')
)

interactive(children=(FloatSlider(value=21.0, description='Minute:', max=45.0, readout_format='.1f', step=1.0)…

<function __main__.plotEventsAtMinute(backgroundImagePath, killDataPath, assistDataPath, deathDataPath, targetMinute, towerDestroyedPath, title)>

In [301]:
print(type(towerData_df))  # This should output <class 'pandas.core.frame.DataFrame'>


<class 'pandas.core.frame.DataFrame'>
